In [102]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):

    if num_layers == 2:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(120, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )

    else:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(120, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )


    return network

In [6]:
PATH = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\good models\copper-sun-2_850.pt"
checkpoint = torch.load(PATH)
# 
# epoch,model_sd,optim_sd,loss_sd = 


In [93]:
model = build_network(128,128,0.3,2)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

Sequential(
  (0): Linear(in_features=120, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=128, out_features=128, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=128, out_features=8, bias=True)
  (7): Softmax(dim=1)
)

In [ ]:
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
pred_inputs_file = open( 'prediction_inputs.npy', 'rb')
data= pickle.load(pred_inputs_file)
race_ids,inputs = zip(*data)
for x in inputs:
    print(x)

In [105]:
scaler = MinMaxScaler()
scaler.fit(inputs)
scaler.transform(inputs)

array([[0.        , 0.        , 0.        , ..., 0.18181818, 0.11111111,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.27272727, 0.27777778,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.72727273, 1.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.27272727, 0.27777778,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [53]:
inputs_df = pd.DataFrame(inputs)
inputs_df.fillna(value=-1, inplace=True)
inputs = inputs_df.values.tolist()
inputs = [[float(i) for i in j] for j in inputs]

X = torch.Tensor(np.array([i for i in inputs]))
X.shape
Xn = nn.functional.normalize(X, dim = 1)
X = Xn

In [ ]:
REBUILD_DATA = True


class GRV_pred:
    # class to store prediction data

    # reading data from pickle
    file = open(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATAprediction_inputs.npy", "rb")
    data = pickle.load(file)
    # seperate out classes from inputs
    raceIDs, inputs = zip(*data)
    # removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1, inplace=True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]

    # data
    pred_data = []

    def make_pred_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.inputs[i]) == 120:
                self.pred_data.append(
                    [
                        np.array(self.inputs[i]),
                    ]
                )
            else:
                adjustedList = self.classes[i] + ([8] * (8 - len(self.classes[i])))
                adjustedListP = self.prices[i] + ([0] * (8 - len(self.prices[i])))
                adjustedListM = self.margins[i] + ([100] * (8 - len(self.margins[i])))
                adjustedListSP = self.margins[i] + ([0] * (8 - len(self.betfairSP[i])))
                self.pred_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(adjustedList),
                        np.array(adjustedListP),
                        np.array(adjustedListM),
                        np.array(adjustedListSP)
                    ]
                )
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save("pred_data.npy", self.pred_data)
        print("excluded = ", excluded)


if REBUILD_DATA:
    grv = GRV_pred()
    grv.make_pred_data()


In [101]:


with torch.no_grad():
    model.eval()
    outputs = model(X)
    print(outputs)
    print(torch.argmax(outputs,1))

tensor([[0.0340, 0.0472, 0.0638, 0.4974, 0.0352, 0.2264, 0.0695, 0.0266],
        [0.0904, 0.0357, 0.0503, 0.3429, 0.0492, 0.3383, 0.0524, 0.0408],
        [0.0684, 0.2530, 0.0695, 0.0828, 0.0397, 0.3650, 0.0762, 0.0454],
        [0.4758, 0.0239, 0.0306, 0.0439, 0.1312, 0.0424, 0.2102, 0.0419],
        [0.0305, 0.0626, 0.0451, 0.1943, 0.0651, 0.0624, 0.5046, 0.0355],
        [0.0374, 0.0321, 0.0580, 0.5199, 0.0320, 0.0802, 0.2033, 0.0371],
        [0.0280, 0.0304, 0.1465, 0.0988, 0.2291, 0.0408, 0.3843, 0.0422],
        [0.1481, 0.0280, 0.0955, 0.1546, 0.1487, 0.0332, 0.3352, 0.0567],
        [0.2744, 0.0195, 0.1578, 0.3533, 0.0248, 0.1031, 0.0490, 0.0182],
        [0.0543, 0.0682, 0.1354, 0.0674, 0.2007, 0.1706, 0.2236, 0.0798],
        [0.0505, 0.2215, 0.0375, 0.3247, 0.0776, 0.1965, 0.0744, 0.0173],
        [0.0583, 0.0256, 0.2015, 0.4515, 0.0654, 0.1442, 0.0362, 0.0173]])
tensor([3, 3, 5, 0, 6, 3, 6, 6, 3, 6, 3, 3])
